In [8]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import re

load_dotenv()

True

In [9]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [14]:
# Function to format the instruction from key facts and event summary
def generate_instruction_from_facts_and_summary(key_facts, event_summary):
    # Format the key facts into a bullet point list
    formatted_facts = '\n'.join(key_facts)

    # Construct the instruction template with the provided facts and event summary
    instruction = f"""
### Talimat:
Aşağıdaki gerçekler verildiğinde:
{formatted_facts}

Ve tarif edilen olay:
{event_summary}

Bu olayın daha geniş etkileri hakkında bir köşe yazısı yazın. Olayın kamu politikalarını, toplumsal tutumları veya gelecekteki gelişmeleri nasıl etkileyebileceğini göz önünde bulundurun.
    """
    return instruction.strip()

# Main function to extract key facts and event summary, then format the instruction
def generate_instruction(article_content):
    # Prepare the conversation state for GPT-4o chat-based completion
    conversation_state = [
        {"role": "system", "content": "Sen, aşağıdaki içeriğin yalnızca gerçeklere dayanan önemli noktalarını ve ana olayın gerçeklere dayanan bir özetini çıkarmakla görevlendirilmiş bir yardımcı asistansın. Orijinal makaleye herhangi bir şekilde atıfta bulunma. 'Makalede belirtiliyor ki' ya da 'Metne göre' gibi ifadelerden kaçın. Gerçek noktaları ve ana olayı tarafsız ve doğrudan bir şekilde sun. Yanıtı şu şekilde biçimlendir:\n\n**Önemli Gerçekler**:\n- Gerçek 1\n- Gerçek 2\n- Gerçek 3\n\n**Ana Olay Özeti**:\n- [Ana olayın kısa özeti]."},
        {"role": "user", "content": f"İçerik burada:\n\n{article_content}\n\nLütfen aşağıdakileri çıkar:\n1. Önemli gerçekler (makalede açıkça belirtilen olgusal noktalar).\n2. Ana olayın gerçeklere dayanan özeti (ana olayı nesnel ve tarafsız bir şekilde tanımla, yorum ya da öznel analiz ekleme)."}
    ]

    # Call GPT-4o chat-based completion API
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=conversation_state,
            max_tokens=600
        )

        # Extract the response from the assistant
        output = response.choices[0].message.content.strip()

        # Use case-insensitive flag via re.IGNORECASE, and clean up the expression
        key_facts_match = re.search(r"\*\*Önemli Gerçekler\*\*([\s\S]*?)\*\*Ana Olay Özeti\*\*", output, re.IGNORECASE)
        event_summary_match = re.search(r"\*\*Ana Olay Özeti\*\*([\s\S]*)", output, re.IGNORECASE)
        
        if key_facts_match:
            # Extract key facts (assuming each fact starts with a hyphen "-")
            key_facts_section = key_facts_match.group(1)
            key_facts = [fact.strip() for fact in key_facts_section.split("\n") if fact.strip().startswith('-')]
        else:
            key_facts = []

        if event_summary_match:
            event_summary = event_summary_match.group(1).strip()

            # Remove any leading colon or unwanted characters before the summary
            event_summary = re.sub(r"^[:\-\s]+", "", event_summary).strip()
        else:
            event_summary = ""

        # Format the final instruction using the key facts and event summary
        instruction = generate_instruction_from_facts_and_summary(key_facts, event_summary)
        return instruction


    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    

# Example usage with one article's content
article_content = """
Alman medyasında Türkiye hakkında bir haber çıktığında hepimiz önce bir irkiliriz.
Acaba terör örgütlerinden hangisini öveceğini düşünürüz.
Bu Almanya'ya karşı bir önyargı değil, yılların getirdiği bir deneyimin sonucudur.
Alman medyası, PKK'dan DHKP/ C'ye, FETÖ'den MLKP'ye nerede Türkiye'nin başına bela olmuş bir terör örgütü varsa onu övmesi ve meşrulaştırmasıyla meşhurdur.
Ancak Alman medyasında Türkiye hakkında yer alan bir haber hepimizi biraz şaşırttı.
Haberi veren Spiegel dergisi, Almanya'nın Türkiye'ye torpido ve güdümlü füze ihracatı için izin verdiğini açıkladı.
Alman hükümetinde bir değişiklik olmadığı, paralel bir evrene de geçmediğimize göre bu oldukça önemli bir haberdi.
Çünkü Almanya merkezli başka bir haber de Siemens firmasının Akkuyu Nükleer Santrali'miz için gerekli ekipmanları yollamadığı gündemdeyken çok sayıda Türkiye düşmanı bakana sahip Alman hükümeti bu izni vermişti.
Cevap aslında oldukça basit ama bir o kadar da önemli: Çünkü artık biz üretiyoruz.
Türk savunma sanayii, Cumhurbaşkanı Erdoğan önderliğinde Türk mühendislerinin insanüstü çabasıyla yurtdışında paramızla bize verilmeyen hangi ürün varsa onu üretmek için var gücüyle çalışıp ambargolarını başlarına çaldı.
O yüzden artık istediğimiz ürünü vermediklerinde çok da dert etmiyoruz.
ABD ve İsrail'in bize vaktiyle İHA vermeyip Türkiye'yi dünyanın en büyük İHA üreticisi yapmalarında olduğu gibi onlarca örnek var.
Mesela Kanada, SİHA'larımız için Elektro-Optik Keşif, Gözetleme ve Hedef Tespit Sistemi CATS'ların satışını yasaklayarak bizi bu sektöre de sokmuştu.
Hatta artık Kanada'dan daha kaliteli ürünler geliştirmiştik.
Ve evet, Kanada da ""ambargosunu"" kaldırmıştı.
Bugün de Almanya'nın bize vaktiyle vermediği torpido ve güdümlü füzeleri üretmeye başladık.
Almanya o yüzden bu satışa izin vererek Türkiye'deki Alman hayranlarını şaşırttı.
Terör örgütü PKK ile mücadele ettiğimiz için parasıyla sattığı tankların parçalarını yollamayan, Türkiye'nin enerji bağımsızlığı için gerekli olan barajları engellemek için her türlü imkânını seferber eden Almanya'dan benzer haberler göreceğiz.
Bu zamana kadar vermek istemediği ne kadar savunma sanayii ürünü varsa şimdi satmak için kendisi çabalayacak.
Bu nokta bizi rehavete sevk etmemeli.
Elbette ki Türk Silahlı Kuvvetleri'mizin ihtiyaçlarının karşılanması gerekiyor ve bu yapılmalı.
Ancak satın aldığımız her ürünün daha iyisini kendi imkânlarımızla üretme amacımızdan bir an bile vazgeçmemeliyiz.
Biz bu hedeften sapmadığımız sürece düne kadar kırk dereden su getiren ""müttefiklerimizin"" kapımızı aşındırdığını göreceğiz.
Çünkü artık biz üretiyoruz.
"""

# Generate the instruction (key facts and event summary)
instruction = generate_instruction(article_content)
print(instruction)

### Talimat:
Aşağıdaki gerçekler verildiğinde:
- Almanya, Türkiye'ye torpido ve güdümlü füze ihracatı için izin verdi.
- Siemens firması, Akkuyu Nükleer Santrali için gerekli ekipmanları teslim etmeme konusunda gündemde.
- Türkiye'nin savunma sanayii, Cumhurbaşkanı Erdoğan önderliğinde ambargolarla karşılaşılan ürünlerin yerli üretimine odaklanmış durumda.
- Türkiye yerli İHA ve SİHA üretiminde dünya çapında tanınan bir konuma ulaştı.

Ve tarif edilen olay:
Almanya'nın Türkiye'ye torpido ve güdümlü füze ihracatına izin vermesi, ülkeler arasındaki ticari ilişkilere dair önemli bir gelişme olarak görülüyor. Türkiye'nin savunma sanayisindeki yerli üretim kapasitesi sayesinde geçmişteki ambargoların üstesinden gelindiği ve stratejik önem taşıyan ürünlerde dışa bağımlılığın azaldığı vurgulanmaktadır.

Bu olayın daha geniş etkileri hakkında bir köşe yazısı yazın. Olayın kamu politikalarını, toplumsal tutumları veya gelecekteki gelişmeleri nasıl etkileyebileceğini göz önünde bulundurun.
